In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pygeohash
!pip install catboost
!pip install lightgbm
!pip install xgboost
!pip install joblib
!pip install optuna

  Preparing metadata (setup.py) ... done
  Created wheel for pygeohash: filename=pygeohash-1.2.0-py2.py3-none-any.whl size=6153 sha256=c2eb7c75d57f569cbf58ae8e1921c6786a8104a6c44997aefd8587dcc3fccd05
  Stored in directory: /root/.cache/pip/wheels/28/ec/b6/beadf7295a623f528507691fb0d471b50d064ae9bbad420b8f
Successfully built pygeohash
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.4 MB/s eta 0:00:00


In [3]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from tqdm import tqdm
import joblib
import math

# import pygeohash as pgh

from concurrent.futures import ThreadPoolExecutor

from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import LinearRegression

from sklearn.model_selection import KFold
from sklearn.metrics import *
from sklearn.preprocessing import LabelEncoder

import gc
gc.collect()
import optuna


import os

In [4]:
path  = "/content/drive/My Drive/signate/SMBC Group GREEN×DATA Challenge 2024/"
# 予測モデルを訓練するためのデータセット
train_df = pd.read_csv(path+'train.csv', index_col=0)

# 予測モデルに推論（予測)させるデータセット
test_df = pd.read_csv(path+'test.csv', index_col=0)

In [5]:
test_df

,FacilityName,Latitude,Longitude,LocationAddress,City,State,ZIP,County,FIPScode,PrimaryNAICS,SecondPrimaryNAICS,IndustryType,TRI_Air_Emissions_10_in_lbs,TRI_Air_Emissions_11_in_lbs,TRI_Air_Emissions_12_in_lbs,TRI_Air_Emissions_13_in_lbs,GHG_Direct_Emissions_10_in_metric_tons,GHG_Direct_Emissions_11_in_metric_tons,GHG_Direct_Emissions_12_in_metric_tons,GHG_Direct_Emissions_13_in_metric_tons
4655,ROGER AOTSCHEGAROW,38.033040,-97.973170,1910 SOUTH BROADACRES ROAD,HUTCHINSON,KS,67501,RENO,20155.0,211112,NaN,"Natural Gas and Natural Gas Liquids Suppliers,...",NaN,NaN,NaN,NaN,88951.390376,65803.021457,2.899329e+04,32965.487915
4656,EPNG STATION 6543 LORDSBURG,32.316030,-108.606800,NaN,LORDSBURG,NM,88045,HIDALGO,35023.0,486210,NaN,Petroleum and Natural Gas Systems,NaN,NaN,NaN,NaN,NaN,NaN,3.119260e+05,268668.069110
4657,OWENS-BROCKWAY GLASS CONTAINER INC PLANT 19,41.251500,-78.742440,3831 ROUTE 219,BROCKPORT,PA,15823,JEFFERSON,42065.0,327213,NaN,Minerals,28275.061385,32075.140635,26445.792090,23385.161510,84908.372871,6212.899604,3.473291e+04,39045.247387
4658,WALDORF CORP (DBA ROCKTENN),44.959660,-93.193410,2250 WABASH AVE,SAINT PAUL,MN,551141895,RAMSEY,27123.0,322130,NaN,Pulp and Paper,37330.205714,42503.897432,43906.102080,42958.306538,49004.603631,52.038452,1.426237e+04,20204.924986
4659,SOLAR TURBINES INC KEARNY MESA,32.818720,-117.125800,4200 RUFFIN RD MZ T-2,SAN DIEGO,CA,92123,SAN DIEGO,6073.0,333611,NaN,Other,45310.722699,39897.800897,40203.428231,39437.099224,84959.769922,15351.359676,4.000654e+04,63935.249120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7158,BEULAH SANITARY LANDFILL,38.651945,-75.914445,ROUTE 331 & ROUTE 16- BEULAH ROAD,HURLOCK,MD,21643,DORCHESTER,24019.0,562212,NaN,Waste,NaN,NaN,NaN,NaN,110523.066755,98954.793233,3.467008e+06,147802.356027
7159,SONORA GAS PLANT,31.361795,-101.776403,NaN,RANKIN,TX,79778,UPTON,48461.0,211112,NaN,Petroleum and Natural Gas Systems,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7160,ASH GROVE CEMENT CO,41.005565,-96.154917,16215 HWY 50,LOUISVILLE,NE,680372881,CASS,31025.0,327310,NaN,Minerals,22956.125554,23592.402994,25069.498286,24560.172725,166520.232511,469732.826538,8.255855e+04,85775.307153
7161,SHOW ME ETHANOL LLC,39.364417,-93.449361,26530 E HWY 24,CARROLLTON,MO,64633,CARROLL,29033.0,325193,NaN,"Other,Waste",124460.895022,140816.449347,93118.705227,130326.946431,71773.803649,936031.894142,3.369613e+05,156640.371706


In [6]:
# すべての列を表示するように設定
pd.set_option('display.max_columns', None)
train_df.columns

Index(['FacilityName', 'Latitude', 'Longitude', 'LocationAddress', 'City',
       'State', 'ZIP', 'County', 'FIPScode', 'PrimaryNAICS',
       'SecondPrimaryNAICS', 'IndustryType', 'TRI_Air_Emissions_10_in_lbs',
       'TRI_Air_Emissions_11_in_lbs', 'TRI_Air_Emissions_12_in_lbs',
       'TRI_Air_Emissions_13_in_lbs', 'GHG_Direct_Emissions_10_in_metric_tons',
       'GHG_Direct_Emissions_11_in_metric_tons',
       'GHG_Direct_Emissions_12_in_metric_tons',
       'GHG_Direct_Emissions_13_in_metric_tons',
       'GHG_Direct_Emissions_14_in_metric_tons'],
      dtype='object')

In [7]:
train_df = train_df.reset_index(drop=True)
test_df  = test_df.reset_index(drop=True)

In [8]:
# del train_df['Unnamed: 0'], test_df['Unnamed: 0']
del train_df['FacilityName'], test_df['FacilityName']
del train_df['LocationAddress'], test_df['LocationAddress']
del train_df['ZIP'], test_df['ZIP']
del train_df['IndustryType'], test_df['IndustryType']
del train_df['SecondPrimaryNAICS'], test_df['SecondPrimaryNAICS']

In [9]:

two_digit_map     = {11: 'Agriculture, Forestry, Fishing and Hunting',
                    21: 'Mining, Quarrying, and Oil and Gas Extraction',
                    22: 'Utilities',
                    23: 'Construction',
                    31: 'Manufacturing',
                    32: 'Manufacturing',
                    33: 'Manufacturing',
                    42: 'Wholesale Trade',
                    44: 'Retail Trade',
                    45: 'Retail Trade',
                    48: 'Transportation and Warehousing',
                    49: 'Transportation and Warehousing',
                    51: 'Information',
                    52: 'Finance and Insurance',
                    53: 'Real Estate and Rental and Leasing',
                    54: 'Professional, Scientific, and Technical Services',
                    55: 'Management of Companies and Enterprises',
                    56: 'Administrative and Support and Waste Management and Remediation Services',
                    61: 'Educational Services',
                    62: 'Health Care and Social Assistance',
                    71: 'Arts, Entertainment, and Recreation',
                    72: 'Accommodation and Food Services',
                    81: 'Other Services (except Public Administration)',
                    92: 'Public Administration'}

In [10]:
train_df['first_two_digit_primary_naics'] = train_df['PrimaryNAICS'].apply(lambda z: str(z)[:2]).astype(int)
test_df['first_two_digit_primary_naics']  = test_df['PrimaryNAICS'].apply(lambda z: str(z)[:2]).astype(int)

train_df['Economic_Sector']               = train_df['first_two_digit_primary_naics'].map(two_digit_map)
test_df['Economic_Sector']                = test_df['first_two_digit_primary_naics'].map(two_digit_map)

del train_df['first_two_digit_primary_naics'], test_df['first_two_digit_primary_naics']

econ_sector_train                         = train_df['Economic_Sector'].values
econ_sector_test                          = test_df['Economic_Sector'].values

In [11]:
test_df.shape

(2508, 16)

In [12]:



def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Radius of Earth in kilometers (mean radius)
    R = 6371.0

    # Calculate the distance
    distance = R * c
    return distance

In [13]:
def calculate_distance(lat1, lon1, lat2, lon2):
    return haversine(lat1, lon1, lat2, lon2)

def get_nearest_distance(lat1, lon1, econ_sector_ref, neighbours=5, train_point=False):
    # Extract the required columns once to NumPy arrays
    latitudes = train_df['Latitude'].values
    longitudes = train_df['Longitude'].values
    ghg_emissions = train_df['GHG_Direct_Emissions_14_in_metric_tons'].values
    econ_sectors = train_df['Economic_Sector'].values

    # Filter NaN emissions directly
    valid_indices = ~np.isnan(ghg_emissions)

    latitudes = latitudes[valid_indices]
    longitudes = longitudes[valid_indices]
    ghg_emissions = ghg_emissions[valid_indices]
    econ_sectors = econ_sectors[valid_indices]

    # Parallelize distance calculation using ThreadPoolExecutor
    with ThreadPoolExecutor() as executor:
        distances = list(executor.map(calculate_distance,
                                     [lat1]*len(latitudes),
                                     [lon1]*len(longitudes),
                                     latitudes,
                                     longitudes))

    # Combine the results into a DataFrame
    near_df = pd.DataFrame({
        'Distance': distances,
        'GHG_emission_14': ghg_emissions,
        'Economic_Sector': econ_sectors
    })


    if train_point:
        near_df.sort_values(by='Distance', inplace=True)
        near_df = near_df.dropna()
        near_df = near_df.iloc[1:].reset_index(drop=True)
    else:
        # Sort distances by 'Distance'
        near_df.sort_values(by='Distance', inplace=True)
        near_df = near_df.dropna()

    # Filter by economic sector
    nearest_locations_econ_sector = near_df[near_df['Economic_Sector'] == econ_sector_ref]

    # Get the top N nearest for both economic sector and overall
    sub_near_econ = nearest_locations_econ_sector.head(neighbours)
    sub_nearest_locations = near_df.head(neighbours)

    # Compute weighted averages
    econ_weighted_average = (sub_near_econ['GHG_emission_14'] / np.where(sub_near_econ['Distance'] == 0, 1, sub_near_econ['Distance']) ).sum()
    near_weighted_average = (sub_nearest_locations['GHG_emission_14'] / np.where(sub_nearest_locations['Distance']==0,1,sub_nearest_locations['Distance'])).sum()

    # Compute regular averages
    econ_average = sub_near_econ['GHG_emission_14'].mean()
    near_average = sub_nearest_locations['GHG_emission_14'].mean()

    return [econ_weighted_average, econ_average, near_weighted_average, near_average]

In [14]:
# neighbours = 5

# # テストデータの距離計算または読み込み
# file_path = f"test_distance_{neighbours}nbrs.csv"
# if os.path.exists(file_path):
#     print(f"{file_path} exists.")
#     test_distance = pd.read_csv(file_path)
# else:
#     output = []
#     for index in tqdm(range(test_df.shape[0])):
#         lat1 = test_df.iloc[index]['Latitude']
#         lon1 = test_df.iloc[index]['Longitude']
#         econ_sector_ref = test_df.iloc[index]['Economic_Sector']
#         x = get_nearest_distance(lat1, lon1, econ_sector_ref, neighbours=neighbours, train_point=False)
#         output.append(x)

#     test_distance = pd.DataFrame(output, columns=['Economy_Sector_Weighted_Avg', 'Economic_Sector_Average', 'Nearest_Weighted_Average', 'Nearest_Average'])
#     test_distance.to_csv(file_path, index=False)

# # トレーニングデータの距離計算または読み込み
# file_path = f"train_distance_{neighbours}nbrs.csv"  # f-string を正しく使用
# if os.path.exists(file_path):
#     print(f"{file_path} exists.")
#     train_distance = pd.read_csv(file_path)
# else:
#     output = []
#     for index in tqdm(range(train_df.shape[0])):
#         lat1 = train_df.iloc[index]['Latitude']
#         lon1 = train_df.iloc[index]['Longitude']
#         econ_sector_ref = train_df.iloc[index]['Economic_Sector']
#         x = get_nearest_distance(lat1, lon1, econ_sector_ref, neighbours=neighbours, train_point=True)
#         output.append(x)

#     train_distance = pd.DataFrame(output, columns=['Economy_Sector_Weighted_Avg', 'Economic_Sector_Average', 'Nearest_Weighted_Average', 'Nearest_Average'])
#     train_distance.to_csv(file_path, index=False)

train_distance = pd.read_csv(path+"train_distance_5nbrs.csv")
test_distance  = pd.read_csv(path+"test_distance_5nbrs.csv")

In [15]:
test_df

,Latitude,Longitude,City,State,County,FIPScode,PrimaryNAICS,TRI_Air_Emissions_10_in_lbs,TRI_Air_Emissions_11_in_lbs,TRI_Air_Emissions_12_in_lbs,TRI_Air_Emissions_13_in_lbs,GHG_Direct_Emissions_10_in_metric_tons,GHG_Direct_Emissions_11_in_metric_tons,GHG_Direct_Emissions_12_in_metric_tons,GHG_Direct_Emissions_13_in_metric_tons,Economic_Sector
0,38.033040,-97.973170,HUTCHINSON,KS,RENO,20155.0,211112,NaN,NaN,NaN,NaN,88951.390376,65803.021457,2.899329e+04,32965.487915,"Mining, Quarrying, and Oil and Gas Extraction"
1,32.316030,-108.606800,LORDSBURG,NM,HIDALGO,35023.0,486210,NaN,NaN,NaN,NaN,NaN,NaN,3.119260e+05,268668.069110,Transportation and Warehousing
2,41.251500,-78.742440,BROCKPORT,PA,JEFFERSON,42065.0,327213,28275.061385,32075.140635,26445.792090,23385.161510,84908.372871,6212.899604,3.473291e+04,39045.247387,Manufacturing
3,44.959660,-93.193410,SAINT PAUL,MN,RAMSEY,27123.0,322130,37330.205714,42503.897432,43906.102080,42958.306538,49004.603631,52.038452,1.426237e+04,20204.924986,Manufacturing
4,32.818720,-117.125800,SAN DIEGO,CA,SAN DIEGO,6073.0,333611,45310.722699,39897.800897,40203.428231,39437.099224,84959.769922,15351.359676,4.000654e+04,63935.249120,Manufacturing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2503,38.651945,-75.914445,HURLOCK,MD,DORCHESTER,24019.0,562212,NaN,NaN,NaN,NaN,110523.066755,98954.793233,3.467008e+06,147802.356027,Administrative and Support and Waste Managemen...
2504,31.361795,-101.776403,RANKIN,TX,UPTON,48461.0,211112,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Mining, Quarrying, and Oil and Gas Extraction"
2505,41.005565,-96.154917,LOUISVILLE,NE,CASS,31025.0,327310,22956.125554,23592.402994,25069.498286,24560.172725,166520.232511,469732.826538,8.255855e+04,85775.307153,Manufacturing
2506,39.364417,-93.449361,CARROLLTON,MO,CARROLL,29033.0,325193,124460.895022,140816.449347,93118.705227,130326.946431,71773.803649,936031.894142,3.369613e+05,156640.371706,Manufacturing


In [16]:
test_distance

,Economy_Sector_Weighted_Avg,Economic_Sector_Average,Nearest_Weighted_Average,Nearest_Average
0,28019.568906,369614.866259,454847.563793,606222.120889
1,1296.575343,75372.922188,26392.933789,299096.231937
2,59857.420688,44340.597367,81533.914173,203831.976280
3,45265.635701,127292.222361,297072.418525,405635.394345
4,8838.742950,70847.235694,617665.034364,290001.288780
...,...,...,...,...
2503,46677.018970,344616.752313,45282.942220,309317.094507
2504,290186.877822,544751.143291,290186.877822,544751.143291
2505,102525.917615,658668.333532,100759.314574,598531.117832
2506,3259.275220,58047.779667,8262.408868,91773.030660


In [17]:
train_df.shape

(4655, 17)

In [18]:
train_df = pd.concat([train_df, train_distance], axis=1)
test_df = pd.concat([test_df, test_distance], axis=1)


In [19]:
def create_features1(df):


    # Year-over-year difference
    df['TRI_Air_Emissions_YoY_Change_11'] = df['TRI_Air_Emissions_11_in_lbs'] - df['TRI_Air_Emissions_10_in_lbs']
    df['TRI_Air_Emissions_YoY_Change_12'] = df['TRI_Air_Emissions_12_in_lbs'] - df['TRI_Air_Emissions_11_in_lbs']
    df['TRI_Air_Emissions_YoY_Change_13'] = df['TRI_Air_Emissions_13_in_lbs'] - df['TRI_Air_Emissions_12_in_lbs']

   # Year-over-year growth rate with NaN protection
    df['TRI_Air_Emissions_Growth_Rate_11'] = np.where(
        df['TRI_Air_Emissions_10_in_lbs'].notna() & (df['TRI_Air_Emissions_10_in_lbs'] != 0),
        (df['TRI_Air_Emissions_11_in_lbs'] - df['TRI_Air_Emissions_10_in_lbs']) / df['TRI_Air_Emissions_10_in_lbs'],
        np.nan
    )

    df['TRI_Air_Emissions_Growth_Rate_12'] = np.where(
        df['TRI_Air_Emissions_11_in_lbs'].notna() & (df['TRI_Air_Emissions_11_in_lbs'] != 0),
        (df['TRI_Air_Emissions_12_in_lbs'] - df['TRI_Air_Emissions_11_in_lbs']) / df['TRI_Air_Emissions_11_in_lbs'],
        np.nan
    )

    df['TRI_Air_Emissions_Growth_Rate_13'] = np.where(
        df['TRI_Air_Emissions_12_in_lbs'].notna() & (df['TRI_Air_Emissions_12_in_lbs'] != 0),
        (df['TRI_Air_Emissions_13_in_lbs'] - df['TRI_Air_Emissions_12_in_lbs']) / df['TRI_Air_Emissions_12_in_lbs'],
        np.nan
    )

    return df

train_df      = create_features1(train_df)
test_df       = create_features1(test_df)
new_features1 = ['TRI_Air_Emissions_YoY_Change_11','TRI_Air_Emissions_YoY_Change_12','TRI_Air_Emissions_YoY_Change_13',
                 'TRI_Air_Emissions_Growth_Rate_11','TRI_Air_Emissions_Growth_Rate_12','TRI_Air_Emissions_Growth_Rate_13']

In [20]:
def create_features2(df):


    # Year-over-year difference
    df['GHG_Direct_Emissions_YoY_Change_11'] = df['GHG_Direct_Emissions_11_in_metric_tons'] - df['GHG_Direct_Emissions_10_in_metric_tons']
    df['GHG_Direct_Emissions_YoY_Change_12'] = df['GHG_Direct_Emissions_12_in_metric_tons'] - df['GHG_Direct_Emissions_11_in_metric_tons']
    df['GHG_Direct_Emissions_YoY_Change_13'] = df['GHG_Direct_Emissions_13_in_metric_tons'] - df['GHG_Direct_Emissions_12_in_metric_tons']

    # Year-over-year growth rate for GHG Direct Emissions with NaN protection
    df['GHG_Direct_Emissions_Growth_Rate_11'] = np.where(
        df['GHG_Direct_Emissions_10_in_metric_tons'].notna() & (df['GHG_Direct_Emissions_10_in_metric_tons'] != 0),
        (df['GHG_Direct_Emissions_11_in_metric_tons'] - df['GHG_Direct_Emissions_10_in_metric_tons']) / df['GHG_Direct_Emissions_10_in_metric_tons'],
        np.nan
    )

    df['GHG_Direct_Emissions_Growth_Rate_12'] = np.where(
        df['GHG_Direct_Emissions_11_in_metric_tons'].notna() & (df['GHG_Direct_Emissions_11_in_metric_tons'] != 0),
        (df['GHG_Direct_Emissions_12_in_metric_tons'] - df['GHG_Direct_Emissions_11_in_metric_tons']) / df['GHG_Direct_Emissions_11_in_metric_tons'],
        np.nan
    )

    df['GHG_Direct_Emissions_Growth_Rate_13'] = np.where(
        df['GHG_Direct_Emissions_12_in_metric_tons'].notna() & (df['GHG_Direct_Emissions_12_in_metric_tons'] != 0),
        (df['GHG_Direct_Emissions_13_in_metric_tons'] - df['GHG_Direct_Emissions_12_in_metric_tons']) / df['GHG_Direct_Emissions_12_in_metric_tons'],
        np.nan
    )

    return df


new_features2 = ['GHG_Direct_Emissions_YoY_Change_11','GHG_Direct_Emissions_YoY_Change_12','GHG_Direct_Emissions_YoY_Change_13',
                 'GHG_Direct_Emissions_Growth_Rate_11','GHG_Direct_Emissions_Growth_Rate_12','GHG_Direct_Emissions_Growth_Rate_13'
                 ]
train_df      = create_features2(train_df)
test_df       = create_features2(test_df)

In [21]:
def create_features3(df):
    df['TRI_to_GHG_Ratio_10'] = np.where(df['GHG_Direct_Emissions_10_in_metric_tons'].notna(),
                                     df['TRI_Air_Emissions_10_in_lbs'] / df['GHG_Direct_Emissions_10_in_metric_tons'],
                                     np.nan)

    df['TRI_to_GHG_Ratio_11'] = np.where(df['GHG_Direct_Emissions_11_in_metric_tons'].notna(),
                                        df['TRI_Air_Emissions_11_in_lbs'] / df['GHG_Direct_Emissions_11_in_metric_tons'],
                                        np.nan)

    df['TRI_to_GHG_Ratio_12'] = np.where(df['GHG_Direct_Emissions_12_in_metric_tons'].notna(),
                                        df['TRI_Air_Emissions_12_in_lbs'] / df['GHG_Direct_Emissions_12_in_metric_tons'],
                                        np.nan)

    df['TRI_to_GHG_Ratio_13'] = np.where(df['GHG_Direct_Emissions_13_in_metric_tons'].notna(),
                                        df['TRI_Air_Emissions_13_in_lbs'] / df['GHG_Direct_Emissions_13_in_metric_tons'],
                                        np.nan)
    return df



train_df      = create_features3(train_df)
test_df       = create_features3(test_df)
new_features3 = ['TRI_to_GHG_Ratio_10','TRI_to_GHG_Ratio_11','TRI_to_GHG_Ratio_12','TRI_to_GHG_Ratio_13']

In [22]:
summary_df = train_df.groupby(['Economic_Sector', 'State']).agg({'GHG_Direct_Emissions_14_in_metric_tons': ['mean', 'median', 'max', 'min','count']})

summary_df.columns = [
                        'GHG_Direct_Emissions_14_in_metric_tons_mean',
                        'GHG_Direct_Emissions_14_in_metric_tons_median',
                        'GHG_Direct_Emissions_14_in_metric_tons_max',
                        'GHG_Direct_Emissions_14_in_metric_tons_min',
                        'GHG_Direct_Emissions_14_in_metric_tons_count'
                     ]

summary_df = summary_df.reset_index()

train_df = train_df.merge(summary_df, on=['Economic_Sector', 'State'], how='left')
test_df  = test_df.merge(summary_df,  on=['Economic_Sector', 'State'], how='left')

In [23]:
test_df.shape

(2508, 41)

In [24]:
train_df.shape

(4655, 42)

In [25]:
merged_df = pd.concat((train_df,test_df),axis=0)
for cols in ['City','State','County','FIPScode','PrimaryNAICS','Economic_Sector']:
    le              = LabelEncoder()
    merged_df[cols] = le.fit_transform(merged_df[cols].values.reshape(-1,1))

train_df = merged_df.iloc[:train_df.shape[0],:]
test_df  = merged_df.iloc[train_df.shape[0]:,:]
train_df.shape,test_df.shape

((4655, 42), (2508, 42))

### ここでもうtest_dfの形が違う

In [26]:
numerical_columns = [
                    'TRI_Air_Emissions_10_in_lbs', 'TRI_Air_Emissions_11_in_lbs',
                    'TRI_Air_Emissions_12_in_lbs', 'TRI_Air_Emissions_13_in_lbs','PrimaryNAICS',
                    'GHG_Direct_Emissions_10_in_metric_tons', 'GHG_Direct_Emissions_11_in_metric_tons',
                    'GHG_Direct_Emissions_12_in_metric_tons', 'GHG_Direct_Emissions_13_in_metric_tons',
                    ]
lat_lon_columns   = ['Latitude','Longitude']
target_columns    = ['GHG_Direct_Emissions_14_in_metric_tons']
categorical_columns = ['City','State','County','FIPScode','PrimaryNAICS','Economic_Sector']
train_aggregations  =  [
                        'GHG_Direct_Emissions_14_in_metric_tons_mean',
                        'GHG_Direct_Emissions_14_in_metric_tons_median',
                        'GHG_Direct_Emissions_14_in_metric_tons_max',
                        'GHG_Direct_Emissions_14_in_metric_tons_min',
                        'GHG_Direct_Emissions_14_in_metric_tons_count'
                     ]
new_features1  = ['TRI_Air_Emissions_YoY_Change_11','TRI_Air_Emissions_YoY_Change_12','TRI_Air_Emissions_YoY_Change_13',
                    'TRI_Air_Emissions_Growth_Rate_11','TRI_Air_Emissions_Growth_Rate_12','TRI_Air_Emissions_Growth_Rate_13']

new_features2   = ['GHG_Direct_Emissions_YoY_Change_11','GHG_Direct_Emissions_YoY_Change_12','GHG_Direct_Emissions_YoY_Change_13',
                  'GHG_Direct_Emissions_Growth_Rate_11','GHG_Direct_Emissions_Growth_Rate_12','GHG_Direct_Emissions_Growth_Rate_13'
                  ]
new_features3   = ['TRI_to_GHG_Ratio_10','TRI_to_GHG_Ratio_11','TRI_to_GHG_Ratio_12','TRI_to_GHG_Ratio_13']
neighbour_feats = ['Economy_Sector_Weighted_Avg','Economic_Sector_Average','Nearest_Weighted_Average','Nearest_Average']


train = train_df[numerical_columns+
                 lat_lon_columns+
                #  categorical_columns+
                 new_features1+['Economic_Sector']
                #  new_features2
               #   new_features3+
                #  train_aggregations+
               #   neighbour_feats
                 ].values
test  = test_df[numerical_columns+
                 lat_lon_columns+
                #  categorical_columns+
                 new_features1+['Economic_Sector']
                #  new_features2
               #   new_features3+
                #  train_aggregations+
               #   neighbour_feats
                 ].values
target = train_df[target_columns].values


In [27]:
len(target)

4655

In [28]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4655 entries, 0 to 4654
Data columns (total 42 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Latitude                                       4553 non-null   float64
 1   Longitude                                      4553 non-null   float64
 2   City                                           4655 non-null   int64  
 3   State                                          4655 non-null   int64  
 4   County                                         4655 non-null   int64  
 5   FIPScode                                       4655 non-null   int64  
 6   PrimaryNAICS                                   4655 non-null   int64  
 7   TRI_Air_Emissions_10_in_lbs                    1635 non-null   float64
 8   TRI_Air_Emissions_11_in_lbs                    1635 non-null   float64
 9   TRI_Air_Emissions_12_in_lbs                    1635 non-n

{'iterations': 864, 'depth': 4, 'learning_rate': 0.02896173229206305, 'l2_leaf_reg': 1.2322248317139508, 'border_count': 254, 'bagging_temperature': 0.14851401382294827, 'random_strength': 0.9638662135548094}

In [30]:

def get_models_trained(train, test, target, num_folds):
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=13)

    oof_predictions = np.zeros(len(train))
    test_predictions = np.zeros(len(test))

    for fold, (train_index, valid_index) in enumerate(kf.split(train, target)):
        print(f"Starting Fold {fold + 1}")
        X_train, X_valid = train[train_index], train[valid_index]
        y_train, y_valid = target[train_index], target[valid_index]

        # モデルと損失の初期化
        model_dict = {}
        loss_dict = {}
        valid_pred_dict = {}

        # モデル1: XGBRegressor
        params1 = {
            "n_estimators": 626,
            "max_depth": 3,
            "random_state": 13,
            "min_child_weight": 0.001190123543553736,
            "learning_rate": 0.010519736270936835,
            "subsample": 0.7304788478701394,
            "colsample_bylevel": 0.604447278915981,
            "colsample_bytree": 0.7616852136157319,
            "reg_alpha": 0.115175569924065,
            "reg_lambda": 0.07155347824929895
        }
        model1 = XGBRegressor(**params1)

        # モデル2: catboost
        params2 = {'iterations': 864,
                   'depth': 4,
                   'learning_rate': 0.02896173229206305,
                   'l2_leaf_reg': 1.2322248317139508,
                   'border_count': 254,
                   'bagging_temperature': 0.14851401382294827,
                   'random_strength': 0.9638662135548094,
                   'random_state': 42}
        model2 = CatBoostRegressor(**params2)

        # モデル3: LGBMRegressor
        # params3 = {
        #     "n_estimators": 56,
        #     "random_state": 13,
        #     "verbose": -1,
        # }
        params3 ={
            "n_estimators": 897,
            "max_depth": 4,
            "min_child_weight": 1.9636282677053687,
            "learning_rate": 0.006151391207761763,
            "subsample": 0.8251910979922186,
            "colsample_bylevel": 0.6454805596196158,
            "colsample_bytree": 0.598042694363472,
            "reg_alpha": 2.1719824223479005,
            "reg_lambda": 0.009192236594241635,
        }
        model3 = XGBRegressor(**params3)

        # モデル1の訓練と予測
        model1.fit(X_train, np.log1p(y_train), verbose=0)
        valid_preds1 = np.expm1(model1.predict(X_valid))
        rmsle1 = np.sqrt(mean_squared_log_error(y_valid, valid_preds1))
        print(f"Fold {fold + 1} RMSLE for model1 = {rmsle1}")

        # モデル2の訓練と予測
        model2.fit(X_train, np.log1p(y_train), verbose=0)
        valid_preds2 = np.expm1(model2.predict(X_valid))
        rmsle2 = np.sqrt(mean_squared_log_error(y_valid, valid_preds2))
        print(f"Fold {fold + 1} RMSLE for model2 = {rmsle2}")

        # モデル3の訓練と予測
        model3.fit(X_train, np.log1p(y_train), verbose=0)
        valid_preds3 = np.expm1(model3.predict(X_valid))
        rmsle3 = np.sqrt(mean_squared_log_error(y_valid, valid_preds3))
        print(f"Fold {fold + 1} RMSLE for model3 = {rmsle3}")

        # 損失とモデルの保存
        loss_dict['model1'] = rmsle1
        loss_dict['model2'] = rmsle2
        loss_dict['model3'] = rmsle3

        model_dict['model1'] = model1
        model_dict['model2'] = model2
        model_dict['model3'] = model3

        # バリデーション予測の保存
        valid_pred_dict['model1'] = valid_preds1
        valid_pred_dict['model2'] = valid_preds2
        valid_pred_dict['model3'] = valid_preds3

        # 平均予測と損失の計算
        valid_preds_mean = np.mean(list(valid_pred_dict.values()), axis=0)
        rmsle_mean = np.sqrt(mean_squared_log_error(y_valid, valid_preds_mean))
        print(f"Fold {fold + 1} Average RMSLE = {rmsle_mean}")

        # 最も損失が小さいモデルの選択
        min_loss_model = min(loss_dict, key=loss_dict.get)
        best_model = model_dict[min_loss_model]
        valid_preds_best_model = np.expm1(best_model.predict(X_valid))
        rmsle_best = np.sqrt(mean_squared_log_error(y_valid, valid_preds_best_model))

        # 平均損失と最良損失の比較
        if rmsle_mean > rmsle_best:
            print(f"The average RMSLE is {rmsle_mean} while the best RMSLE is {rmsle_best} and we proceed with the model with best RMSLE")
            oof_predictions[valid_index] = valid_preds_best_model
            test_preds = best_model.predict(test)
            test_predictions += (test_preds) / kf.n_splits
        else:
            print(f"The average RMSLE is {rmsle_mean} while the best RMSLE is {rmsle_best} and we proceed with averaging of all models")
            output_predictions = [model.predict(test) for model in model_dict.values()]
            output_preds = np.mean(output_predictions, axis=0)
            test_predictions += output_preds / kf.n_splits
            oof_predictions[valid_index] = valid_preds_mean

        # メモリの解放
        del X_train, X_valid, y_train, y_valid, model1, model2, model3
        gc.collect()

        print('---------------\n')

    # 最終的なOOF RMSLEの計算
    final_RMSLE = np.sqrt(mean_squared_log_error(target, oof_predictions))
    print(f"OOF RMSLE = {final_RMSLE}")

    # テスト予測の逆変換
    final_test_predictions = np.expm1(test_predictions)

    return oof_predictions, final_test_predictions

In [31]:
oof_predictions,test_preds = get_models_trained(train,test,target,30)

ストリーミング出力は最後の 5000 行に切り捨てられました。
231:	learn: 0.7364357	total: 586ms	remaining: 1.6s
232:	learn: 0.7363592	total: 589ms	remaining: 1.59s
233:	learn: 0.7362468	total: 591ms	remaining: 1.59s
234:	learn: 0.7360231	total: 593ms	remaining: 1.59s
235:	learn: 0.7358822	total: 596ms	remaining: 1.58s
236:	learn: 0.7357914	total: 598ms	remaining: 1.58s
237:	learn: 0.7355910	total: 600ms	remaining: 1.58s
238:	learn: 0.7353938	total: 603ms	remaining: 1.58s
239:	learn: 0.7352390	total: 605ms	remaining: 1.57s
240:	learn: 0.7351068	total: 608ms	remaining: 1.57s
241:	learn: 0.7350210	total: 610ms	remaining: 1.57s
242:	learn: 0.7349591	total: 612ms	remaining: 1.56s
243:	learn: 0.7348522	total: 614ms	remaining: 1.56s
244:	learn: 0.7346975	total: 617ms	remaining: 1.56s
245:	learn: 0.7346358	total: 619ms	remaining: 1.55s
246:	learn: 0.7345921	total: 621ms	remaining: 1.55s
247:	learn: 0.7344285	total: 625ms	remaining: 1.55s
248:	learn: 0.7342565	total: 627ms	remaining: 1.55s
249:	learn: 0.7341337	total: 630m

In [32]:

# Step 1: Optuna の Objective 関数
def objective(trial, X, y, num_folds):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-3, 10.0, log=True),
        'border_count': trial.suggest_int('border_count', 32, 255),
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'random_strength': trial.suggest_float('random_strength', 0.0, 1.0),
        'od_type': 'Iter',
        'od_wait': 20,
        'verbose': 0,
        'random_state': 13
    }

    kf = KFold(n_splits=num_folds, shuffle=True, random_state=13)
    oof_preds = np.zeros(len(X))

    for train_idx, valid_idx in kf.split(X, y):
        X_train, X_valid = X[train_idx], X[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]

        model = CatBoostRegressor(**params)
        model.fit(X_train, np.log1p(y_train), eval_set=(X_valid, y_valid), early_stopping_rounds=50, verbose=False)

        preds = np.expm1(model.predict(X_valid))
        rmsle = np.sqrt(mean_squared_log_error(y_valid, preds))
        oof_preds[valid_idx] = preds

    final_rmsle = np.sqrt(mean_squared_log_error(y, oof_preds))
    return final_rmsle


In [33]:

# Step 2: Optuna を用いたハイパーパラメータチューニング
def tune_catboost(X, y, num_folds=5, n_trials=200):
    study = optuna.create_study(direction='minimize')
    study.optimize(lambda trial: objective(trial, X, y, num_folds), n_trials=n_trials)
    print("Best trial:")
    trial = study.best_trial
    print(f"  Value: {trial.value}")
    print("  Params: ")
    for key, value in trial.params.items():
        print(f"    {key}: {value}")
    return trial.params

In [34]:

# Step 3: K-Fold Cross-Validation を用いたモデル訓練と予測
def get_models_trained(train, test, target, num_folds, catboost_params):
    kf = KFold(n_splits=num_folds, shuffle=True, random_state=13)

    oof_predictions = np.zeros(len(train))
    test_predictions = np.zeros(len(test))

    for fold, (train_index, valid_index) in enumerate(kf.split(train, target)):
        print(f"Starting Fold {fold + 1}")
        X_train, X_valid = train[train_index], train[valid_index]
        y_train, y_valid = target[train_index], target[valid_index]

        # CatBoostRegressor のインスタンス作成
        model = CatBoostRegressor(**catboost_params, random_state=13, verbose=0)

        # モデルの訓練
        model.fit(X_train, np.log1p(y_train), eval_set=(X_valid, y_valid), early_stopping_rounds=50, verbose=False)

        # バリデーションデータに対する予測
        valid_preds = np.expm1(model.predict(X_valid))
        rmsle = np.sqrt(mean_squared_log_error(y_valid, valid_preds))
        print(f"Fold {fold + 1} RMSLE = {rmsle}")

        # OOF 予測の保存
        oof_predictions[valid_index] = valid_preds

        # テストデータに対する予測の蓄積
        test_preds = np.expm1(model.predict(test))
        test_predictions += test_preds / num_folds

        # メモリの解放
        del X_train, X_valid, y_train, y_valid, model
        gc.collect()

        print('---------------\n')

    # 最終的な OOF RMSLE の計算
    final_RMSLE = np.sqrt(mean_squared_log_error(target, oof_predictions))
    print(f"OOF RMSLE = {final_RMSLE}")

    return oof_predictions, test_predictions


In [35]:
    # # Step 1: ハイパーパラメータの最適化
    # best_params = tune_catboost(train, target, num_folds=30, n_trials=200)

    # # Step 2: モデルの訓練と予測
    # oof_preds, test_preds = get_models_trained(train, test, target, num_folds=30, catboost_params=best_params)


In [36]:
submit = pd.read_csv(path+'sample_submission.csv',  header = None)
submit

,0,1
0,4655,249574.303827
1,4656,249574.303827
2,4657,249574.303827
3,4658,249574.303827
4,4659,249574.303827
...,...,...
2503,7158,249574.303827
2504,7159,249574.303827
2505,7160,249574.303827
2506,7161,249574.303827


In [37]:
test_preds.shape

(2508,)

In [41]:
submit [ 1 ]  =  test_preds
submit.to_csv('SMBC_13.csv', index = False)


In [42]:
submit

,0,1
0,4655,4.405101e+04
1,4656,3.184454e+05
2,4657,3.975497e+04
3,4658,2.687447e+04
4,4659,4.725183e+04
...,...,...
2503,7158,5.112241e+05
2504,7159,1.030941e+05
2505,7160,1.469505e+05
2506,7161,2.522208e+05


In [43]:
import time
print(time.ctime())

Tue Nov 19 10:14:21 2024
